In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
file_path = '/content/anime.csv'
anime_data = pd.read_csv(file_path)
anime_data

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
...,...,...,...,...,...,...,...
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211
12290,5543,Under World,Hentai,OVA,1,4.28,183
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,219
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175


## Data Preprocessing

In [3]:
df=anime_data[['anime_id','genre','rating']]
df

,anime_id,genre,rating
0,32281,"Drama, Romance, School, Supernatural",9.37
1,5114,"Action, Adventure, Drama, Fantasy, Magic, Mili...",9.26
2,28977,"Action, Comedy, Historical, Parody, Samurai, S...",9.25
3,9253,"Sci-Fi, Thriller",9.17
4,9969,"Action, Comedy, Historical, Parody, Samurai, S...",9.16
...,...,...,...
12289,9316,Hentai,4.15
12290,5543,Hentai,4.28
12291,5621,Hentai,4.88
12292,6133,Hentai,4.98


In [4]:
df.head()

,anime_id,genre,rating
0,32281,"Drama, Romance, School, Supernatural",9.37
1,5114,"Action, Adventure, Drama, Fantasy, Magic, Mili...",9.26
2,28977,"Action, Comedy, Historical, Parody, Samurai, S...",9.25
3,9253,"Sci-Fi, Thriller",9.17
4,9969,"Action, Comedy, Historical, Parody, Samurai, S...",9.16


In [50]:
df.tail()

,anime_id,genre,rating
12289,9316,Hentai,4.15
12290,5543,Hentai,4.28
12291,5621,Hentai,4.88
12292,6133,Hentai,4.98
12293,26081,Hentai,5.46


In [6]:
df.shape

(12294, 3)

In [7]:
df.isnull().sum()

,0
anime_id,0
genre,62
rating,230


In [8]:
df.fillna({'genre': 'Unknown', 'rating': anime_data['rating'].mean()}, inplace=True)


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   genre     12294 non-null  object 
 2   rating    12294 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 288.3+ KB


In [10]:
df.describe()

,anime_id,rating
count,12294.000000,12294.000000
mean,14058.221653,6.473902
std,11455.294701,1.017096
min,1.000000,1.670000
25%,3484.250000,5.900000
50%,10260.500000,6.550000
75%,24794.500000,7.170000
max,34527.000000,10.000000


In [11]:
len(df.anime_id.unique())

12294

In [12]:
df.anime_id.unique()

array([32281,  5114, 28977, ...,  5621,  6133, 26081])

In [13]:
len(df.genre.unique())

3265

In [14]:
df.rating.unique()

array([ 9.37      ,  9.26      ,  9.25      ,  9.17      ,  9.16      ,
        9.15      ,  9.13      ,  9.11      ,  9.1       ,  9.06      ,
        9.05      ,  9.04      ,  8.98      ,  8.93      ,  8.92      ,
        8.88      ,  8.84      ,  8.83      ,  8.82      ,  8.81      ,
        8.8       ,  8.78      ,  8.77      ,  8.76      ,  8.75      ,
        8.74      ,  8.73      ,  8.72      ,  8.71      ,  8.69      ,
        8.68      ,  8.67      ,  8.66      ,  8.65      ,  8.64      ,
        8.62      ,  8.61      ,  8.6       ,  8.59      ,  8.58      ,
        8.57      ,  8.56      ,  8.55      ,  8.54      ,  8.53      ,
        8.52      ,  8.51      ,  8.5       ,  8.49      ,  8.48      ,
        8.47      ,  8.46      ,  8.45      ,  8.44      ,  8.43      ,
        8.42      ,  8.41      ,  8.4       ,  8.39      ,  8.38      ,
        8.37      ,  8.36      ,  8.35      ,  8.34      ,  8.33      ,
        8.32      ,  8.31      ,  8.3       ,  8.29      ,  8.28

In [15]:
df.groupby('genre')['rating'].mean().sort_values(ascending=False)

,rating
genre,
"Action, Adventure, Drama, Fantasy, Magic, Military, Shounen",9.26
"Drama, Fantasy, Romance, Slice of Life, Supernatural",9.06
"Drama, School, Shounen",9.05
"Adventure, Drama, Supernatural",8.93
"Drama, Music, Romance, School, Shounen",8.92
...,...
"Action, Demons, Seinen, Super Power",3.32
"Kids, Mecha",3.26
"Action, Adventure, Sci-Fi, Space, Super Power",3.25


In [16]:
df.groupby('genre')['rating'].count().sort_values(ascending=False)

,rating
genre,
Hentai,823
Comedy,523
Music,301
Kids,199
"Comedy, Slice of Life",179
...,...
"Action, Comedy, Fantasy, Magic, Shounen",1
"Action, Comedy, Fantasy, Magic, Slice of Life",1
"Adventure, Super Power",1


## Feature Extraction

In [17]:
df=anime_data[['anime_id','genre','rating']]
df

,anime_id,genre,rating
0,32281,"Drama, Romance, School, Supernatural",9.37
1,5114,"Action, Adventure, Drama, Fantasy, Magic, Mili...",9.26
2,28977,"Action, Comedy, Historical, Parody, Samurai, S...",9.25
3,9253,"Sci-Fi, Thriller",9.17
4,9969,"Action, Comedy, Historical, Parody, Samurai, S...",9.16
...,...,...,...
12289,9316,Hentai,4.15
12290,5543,Hentai,4.28
12291,5621,Hentai,4.88
12292,6133,Hentai,4.98


## Recommendation System

## Cosine Similarity

## Item Based

In [18]:
df1=df.pivot_table(index='genre',columns='anime_id',values='rating')
df1.fillna(0,axis=1,inplace=True)

In [19]:
df1

anime_id,1,5,6,7,8,15,16,17,18,19,...,34412,34447,34453,34464,34475,34476,34490,34503,34514,34519
genre,,,,,,,,,,,,,,,,,,,,,
Action,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,6.64,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Action, Adventure",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Action, Adventure, Cars, Comedy, Sci-Fi, Shounen",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Action, Adventure, Cars, Mecha, Sci-Fi, Shounen, Sports",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Action, Adventure, Cars, Sci-Fi",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Super Power, Supernatural, Vampire",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Supernatural,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Thriller,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
arr=cosine_similarity(df1)
arr

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [21]:
df2=pd.DataFrame(arr)
df2

,0,1,2,3,4,5,6,7,8,9,...,3219,3220,3221,3222,3223,3224,3225,3226,3227,3228
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3224,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3225,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3226,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3227,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [22]:
df1.index[1]

'Action, Adventure'

In [23]:
df1.index

Index(['Action', 'Action, Adventure',
       'Action, Adventure, Cars, Comedy, Sci-Fi, Shounen',
       'Action, Adventure, Cars, Mecha, Sci-Fi, Shounen, Sports',
       'Action, Adventure, Cars, Sci-Fi', 'Action, Adventure, Comedy',
       'Action, Adventure, Comedy, Demons, Drama, Ecchi, Horror, Mystery, Romance, Sci-Fi',
       'Action, Adventure, Comedy, Demons, Fantasy, Magic',
       'Action, Adventure, Comedy, Demons, Fantasy, Magic, Romance, Shounen, Supernatural',
       'Action, Adventure, Comedy, Demons, Fantasy, Martial Arts, Shounen, Super Power',
       ...
       'Slice of Life', 'Slice of Life, Space', 'Slice of Life, Supernatural',
       'Space', 'Sports', 'Super Power, Supernatural, Vampire', 'Supernatural',
       'Thriller', 'Vampire', 'Yaoi'],
      dtype='object', name='genre', length=3229)

In [24]:
for i in enumerate(range(10,15)):
    print(i)

(0, 10)
(1, 11)
(2, 12)
(3, 13)
(4, 14)


In [25]:
sorted(list(enumerate(arr[0])),key=lambda x: x[1],reverse=True)[1:6]

[(1, 0.0), (2, 0.0), (3, 0.0), (4, 0.0), (5, 0.0)]

In [26]:
similarities=sorted(list(enumerate(arr[0])),key=lambda x:x[1],reverse=True)[1:6]
similarities

[(1, 0.0), (2, 0.0), (3, 0.0), (4, 0.0), (5, 0.0)]

In [27]:
np.where('Action, Adventure, Cars, Mecha, Sci-Fi, Shounen, Sports'==df1.index)[0][0]

3

In [28]:
def recommend_movies(movie_list):
    if movie_list in df1.index:
        index= np.where(movie_list==df1.index)[0][0]
        similar_movie= sorted(list(enumerate(arr[index])),key=lambda x: x[1],reverse=True)[1:6]
        print(f'recommended movies of {movie_list}')
        print('-'*10)
        for movie in similar_movie:
            print(df1.index[movie[0]])
    else:
        print('Move is not in the list')

In [29]:
recommend_movies('Action, Adventure, Cars, Sci-Fi')

recommended movies of Action, Adventure, Cars, Sci-Fi
----------
Action
Action, Adventure
Action, Adventure, Cars, Comedy, Sci-Fi, Shounen
Action, Adventure, Cars, Mecha, Sci-Fi, Shounen, Sports
Action, Adventure, Comedy


In [30]:
recommend_movies('Action, Adventure, Comedy, Demons, Fantasy, Martial Arts, Shounen, Super Power')

recommended movies of Action, Adventure, Comedy, Demons, Fantasy, Martial Arts, Shounen, Super Power
----------
Action
Action, Adventure
Action, Adventure, Cars, Comedy, Sci-Fi, Shounen
Action, Adventure, Cars, Mecha, Sci-Fi, Shounen, Sports
Action, Adventure, Cars, Sci-Fi


## User Based

In [31]:
df2=df.pivot_table(index='anime_id',columns='genre',values='rating')
df2.fillna(0,axis=1,inplace=True)

In [32]:
len(df.anime_id.unique())

12294

In [33]:
genre_dummies = df['genre'].str.get_dummies(sep=', ')
feature_matrix = pd.concat([df['anime_id'], genre_dummies], axis=1)

In [34]:
genre_dummies

,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,...,Shounen Ai,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Vampire,Yaoi,Yuri
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,1,1,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12289,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12290,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12291,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12292,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
arr=cosine_similarity(genre_dummies)
np.fill_diagonal(arr,'nan')

In [36]:
arr

array([[       nan, 0.18898224, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.18898224,        nan, 0.28571429, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.28571429,        nan, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ...,        nan, 1.        ,
        1.        ],
       [0.        , 0.        , 0.        , ..., 1.        ,        nan,
        1.        ],
       [0.        , 0.        , 0.        , ..., 1.        , 1.        ,
               nan]])

In [37]:
df_new=pd.DataFrame(arr)
np.fill_diagonal(arr,'nan')

In [38]:
df_new

,0,1,2,3,4,5,6,7,8,9,...,12284,12285,12286,12287,12288,12289,12290,12291,12292,12293
0,NaN,0.188982,0.000000,0.000000,0.000000,0.447214,0.000000,0.250000,0.000000,0.000000,...,0.0,0.288675,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.188982,NaN,0.285714,0.000000,0.285714,0.338062,0.566947,0.377964,0.285714,0.285714,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.285714,NaN,0.267261,1.000000,0.338062,0.377964,0.188982,1.000000,1.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.267261,NaN,0.267261,0.000000,0.000000,0.353553,0.267261,0.267261,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.285714,1.000000,0.267261,NaN,0.338062,0.377964,0.188982,1.000000,1.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12289,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.0,0.577350,1.0,1.0,1.0,NaN,1.0,1.0,1.0,1.0
12290,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.0,0.577350,1.0,1.0,1.0,1.0,NaN,1.0,1.0,1.0
12291,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.0,0.577350,1.0,1.0,1.0,1.0,1.0,NaN,1.0,1.0
12292,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.0,0.577350,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0


In [39]:
df_new.index=df.anime_id.unique()
df_new.columns=df.anime_id.unique()

In [40]:
df_new.index

Index([32281,  5114, 28977,  9253,  9969, 32935, 11061,   820, 15335, 15417,
       ...
       26031, 34399, 10368,  9352,  5541,  9316,  5543,  5621,  6133, 26081],
      dtype='int64', length=12294)

In [41]:
df_new.columns

Index([32281,  5114, 28977,  9253,  9969, 32935, 11061,   820, 15335, 15417,
       ...
       26031, 34399, 10368,  9352,  5541,  9316,  5543,  5621,  6133, 26081],
      dtype='int64', length=12294)

In [42]:
df_new

,32281,5114,28977,9253,9969,32935,11061,820,15335,15417,...,26031,34399,10368,9352,5541,9316,5543,5621,6133,26081
32281,NaN,0.188982,0.000000,0.000000,0.000000,0.447214,0.000000,0.250000,0.000000,0.000000,...,0.0,0.288675,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5114,0.188982,NaN,0.285714,0.000000,0.285714,0.338062,0.566947,0.377964,0.285714,0.285714,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28977,0.000000,0.285714,NaN,0.267261,1.000000,0.338062,0.377964,0.188982,1.000000,1.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9253,0.000000,0.000000,0.267261,NaN,0.267261,0.000000,0.000000,0.353553,0.267261,0.267261,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9969,0.000000,0.285714,1.000000,0.267261,NaN,0.338062,0.377964,0.188982,1.000000,1.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9316,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.0,0.577350,1.0,1.0,1.0,NaN,1.0,1.0,1.0,1.0
5543,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.0,0.577350,1.0,1.0,1.0,1.0,NaN,1.0,1.0,1.0
5621,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.0,0.577350,1.0,1.0,1.0,1.0,1.0,NaN,1.0,1.0
6133,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.0,0.577350,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0


In [43]:
df_new.idxmax()

,0
32281,547
5114,121
28977,9969
9253,11577
9969,28977
...,...
9316,11879
5543,11879
5621,11879
6133,11879


In [44]:
df[(df.anime_id==5114) | (df.anime_id==121)]

,anime_id,genre,rating
1,5114,"Action, Adventure, Drama, Fantasy, Magic, Mili...",9.26
200,121,"Action, Adventure, Comedy, Drama, Fantasy, Mag...",8.33


In [45]:
df[(df.anime_id==28977) | (df.anime_id==9969)]

,anime_id,genre,rating
2,28977,"Action, Comedy, Historical, Parody, Samurai, S...",9.25
4,9969,"Action, Comedy, Historical, Parody, Samurai, S...",9.16


In [46]:
df[(df.anime_id==5621) | (df.anime_id==11879)]

,anime_id,genre,rating
11114,11879,Hentai,7.89
12291,5621,Hentai,4.88


## Evaluation

In [52]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357285 sha256=eef171efea67942f9aed29b4e9f3d7c9375ac9d65ab221208867577557aea36b
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [56]:
from surprise import Reader, Dataset, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy, KNNBasic

In [94]:
from sklearn.model_selection import train_test_split

# Assuming 'user_id' and 'anime_id' are the columns for user-item interactions
X = df[['anime_id']]  # Features
y = df['rating']  # Target variable

# Split the dataset into training and testing sets (80% train, 20% test)
train_data, test_data= train_test_split(df, test_size=0.2, random_state=42)

In [48]:
train_data

,anime_id,genre,rating
3013,5342,"Action, Mecha, Supernatural",7.10
4253,9581,"Comedy, Ecchi, School",6.77
9791,9810,Comedy,6.75
2629,1539,"Romance, Shounen, Sports",7.21
4608,4439,"Action, Martial Arts, Sports",6.68
...,...,...,...
11964,4638,"Hentai, Romance",5.82
5191,5272,Adventure,6.53
5390,1262,"Adventure, Mecha, Military, Sci-Fi, Shounen, S...",6.47
860,22819,"Music, School, Shoujo, Slice of Life",7.79


In [49]:
test_data

,anime_id,genre,rating
6329,17209,Kids,6.17
2167,173,"Comedy, Drama, Fantasy, Mystery, Shounen, Supe...",7.34
2882,3616,"Action, Comedy, Ecchi, Super Power",7.14
4700,18799,"Action, Music, Seinen, Supernatural",6.66
7258,18831,"Dementia, Horror, Music",5.60
...,...,...,...
9652,26305,"Demons, Drama, Kids",5.75
549,30311,"Comedy, Sports",7.98
7550,30327,Dementia,5.31
5210,13137,"Comedy, Ecchi",6.52


In [81]:
reader = Reader(rating_scale=(1, 10))

In [95]:
anime_data

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
...,...,...,...,...,...,...,...
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211
12290,5543,Under World,Hentai,OVA,1,4.28,183
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,219
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175


In [96]:
data=anime_data.drop(columns=['type','episodes','name','genre'])

In [97]:
data

,anime_id,rating,members
0,32281,9.37,200630
1,5114,9.26,793665
2,28977,9.25,114262
3,9253,9.17,673572
4,9969,9.16,151266
...,...,...,...
12289,9316,4.15,211
12290,5543,4.28,183
12291,5621,4.88,219
12292,6133,4.98,175


In [98]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [99]:
train_data

,anime_id,rating,members
3013,5342,7.10,68608
4253,9581,6.77,21462
9791,9810,6.75,169
2629,1539,7.21,1513
4608,4439,6.68,603
...,...,...,...
11964,4638,5.82,695
5191,5272,6.53,252
5390,1262,6.47,6760
860,22819,7.79,2813


In [106]:
!pip install scikit-surprise

from surprise.model_selection import train_test_split
from surprise import Reader, Dataset

# Assuming 'data' is your pandas DataFrame
reader = Reader(rating_scale=(1, 10))
data_surprise = Dataset.load_from_df(data[['anime_id', 'rating','members']], reader)

# Use Surprise's train_test_split
trainset, testset = train_test_split(data_surprise, test_size=0.2)

In [107]:
model = SVD()
model.fit(trainset)

In [109]:
predictions = model.test(testset)
predictions

[Prediction(uid=11001, iid=7.96, r_ui=10244.0, est=10, details={'was_impossible': False}),
 Prediction(uid=22845, iid=6.25, r_ui=44.0, est=10, details={'was_impossible': False}),
 Prediction(uid=2234, iid=6.56, r_ui=1358.0, est=10, details={'was_impossible': False}),
 Prediction(uid=30458, iid=7.71, r_ui=70635.0, est=10, details={'was_impossible': False}),
 Prediction(uid=570, iid=7.87, r_ui=62464.0, est=10, details={'was_impossible': False}),
 Prediction(uid=8654, iid=5.99, r_ui=4098.0, est=10, details={'was_impossible': False}),
 Prediction(uid=8547, iid=6.47, r_ui=537.0, est=10, details={'was_impossible': False}),
 Prediction(uid=32146, iid=4.67, r_ui=83.0, est=10, details={'was_impossible': False}),
 Prediction(uid=7364, iid=6.72, r_ui=736.0, est=10, details={'was_impossible': False}),
 Prediction(uid=21469, iid=8.12, r_ui=5712.0, est=10, details={'was_impossible': False}),
 Prediction(uid=28577, iid=8.0, r_ui=37.0, est=10, details={'was_impossible': False}),
 Prediction(uid=1453, 

In [110]:
from collections import defaultdict

def get_top_n(predictions, n=10):
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Sort the predictions for each user and retrieve the n highest ones
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

top_n = get_top_n(predictions, n=10)

In [112]:
top_n

defaultdict(list,
            {11001: [(7.96, 10)],
             22845: [(6.25, 10)],
             2234: [(6.56, 10)],
             30458: [(7.71, 10)],
             570: [(7.87, 10)],
             8654: [(5.99, 10)],
             8547: [(6.47, 10)],
             32146: [(4.67, 10)],
             7364: [(6.72, 10)],
             21469: [(8.12, 10)],
             28577: [(8.0, 10)],
             1453: [(8.25, 10)],
             6512: [(7.39, 10)],
             12503: [(6.79, 10)],
             2794: [(6.2, 10)],
             2944: [(5.63, 10)],
             30056: [(6.63, 10)],
             28097: [(6.0, 10)],
             2249: [(5.6, 10)],
             2704: [(6.82, 10)],
             20365: [(6.36, 10)],
             1527: [(6.57, 10)],
             23425: [(7.13, 10)],
             1956: [(6.96, 10)],
             10218: [(7.91, 10)],
             31667: [(3.0, 10)],
             29777: [(4.58, 10)],
             4451: [(6.35, 10)],
             23855: [(5.25, 10)],
             880

In [140]:
predictions_df = pd.DataFrame(predictions, columns=['user_id', 'anime_id', 'true_rating', 'rating', 'details'])

predictions_df

,user_id,anime_id,true_rating,rating,details
0,11001,7.96,10244.0,10,{'was_impossible': False}
1,22845,6.25,44.0,10,{'was_impossible': False}
2,2234,6.56,1358.0,10,{'was_impossible': False}
3,30458,7.71,70635.0,10,{'was_impossible': False}
4,570,7.87,62464.0,10,{'was_impossible': False}
...,...,...,...,...,...
2454,8129,8.24,108053.0,10,{'was_impossible': False}
2455,29908,4.69,152.0,10,{'was_impossible': False}
2456,8426,7.80,53426.0,10,{'was_impossible': False}
2457,24627,7.59,33816.0,10,{'was_impossible': False}


In [152]:
predictions_df = pd.DataFrame(predictions, columns=['user_id', 'anime_id', 'true_rating', 'rating', 'details'])

threshold = 4
predictions_df['relevant'] = (predictions_df['true_rating'] >= threshold).astype(int)
# Access the correct column name ('rating')
predictions_df['predicted_relevant'] = (predictions_df['rating'] >= threshold).astype(int)

In [153]:
predictions_df = pd.DataFrame(predictions, columns=['user_id', 'item_id', 'true_rating', 'estimated_rating', 'details'])
# predictions_df

threshold = 4
predictions_df['relevant'] = (predictions_df['true_rating'] >= threshold).astype(int)
predictions_df['predicted_relevant'] = (predictions_df['estimated_rating'] >= threshold).astype(int)

# Calculate precision and recall
y_true = predictions_df['relevant']
y_pred = predictions_df['predicted_relevant']
precision = (y_true * y_pred).sum() / y_pred.sum()
recall = (y_true * y_pred).sum() / y_true.sum()
accuracy=(y_true==y_pred).sum()/len(y_true)
f1_score=2*(precision*recall)/(precision+recall)

In [148]:
precision

1.0

In [149]:
recall

1.0

In [150]:
accuracy

1.0

In [151]:
f1_score

1.0

Precision:It indicates the proportion of recommended items that were relevant.

Recall:It indicates the proportion of relevant items that were recommended.

Accuracy:It measures the overall correctness of the recommendations.

##Analyze Performance

1.Review Metrics: Assess precision, recall, and F1 scores to gauge the model's performance.

2.Feedback Loop: Gather user feedback to understand satisfaction with recommendations.

3.Adjustments: Experiment with feature selection, similarity thresholds, and user segmentation to improve performance.

## Interview Questions

## 1. Can you explain the difference between user-based and item-based collaborative filtering?


## User-Based Collaborative Filtering

Concept: This approach recommends items to a user based on the preferences of similar users. The idea is that if User A and User B have a high similarity in their preferences (e.g., they rated the same items similarly), then User A may enjoy items that User B liked but User A hasn't yet rated.

Process:

1.Identify Similar Users: Calculate similarity scores between users using metrics like cosine similarity or Pearson correlation.

2.Generate Recommendations: For a target user, find the top similar users and recommend items they liked but the target user hasn't interacted with yet.

Advantages:

1.Can adapt quickly to changing user preferences.

2.Easy to understand and explain.

Disadvantages:

1.Suffers from the "cold start" problem for new users with no ratings.

2.Scalability issues with large datasets, as finding similar users can be computationally intensive.

## Item-Based Collaborative Filtering

Concept: Instead of finding similar users, this approach recommends items based on their similarity to items the user has already liked. If an item is similar to items a user has rated highly, it is recommended to them.

Process:

1.Identify Similar Items: Calculate similarity scores between items based on user ratings (e.g., how often two items are rated by the same users).

2.Generate Recommendations: For a target item that the user has rated, recommend other items that are most similar to it.

Advantages:

1.More stable over time; item similarities tend to be more consistent than user preferences.

2.Often scales better because you only need to compute item similarities once.

Disadvantages:

1.The cold start problem still exists for new items with no ratings.

2.Recommendations may lack novelty if similar items are already known to the user.


##Summary

User-Based: Focuses on finding similar users to recommend items based on their preferences.
    
Item-Based: Focuses on finding similar items based on user ratings and recommends those items to users.

## 2. What is collaborative filtering, and how does it work?

Collaborative filtering is a popular technique used in recommendation systems to suggest items to users based on the preferences and behaviors of other users. The fundamental idea is that users who agreed in the past will agree in the future, allowing the system to predict a user's preferences for items they have not yet interacted with.

##How Collaborative Filtering Works

Collaborative filtering can be divided into two main categories: user-based and item-based filtering, as previously discussed. Here’s a closer look at how it works in general:

1. Data Collection

Collaborative filtering relies on user-item interaction data, which can include:

User ratings (e.g., 1 to 5 stars).
Purchase history (e.g., items bought).
User behavior data (e.g., clicks, views).
This data is typically represented in a user-item matrix, where rows represent users, columns represent items, and the cells contain the interactions (ratings, purchases, etc.).

2. Similarity Calculation

User-Based Collaborative Filtering:

Calculate the similarity between users using metrics like:
Cosine Similarity: Measures the cosine of the angle between two non-zero vectors (user profiles).
Pearson Correlation: Measures the linear correlation between two sets of ratings.
    
Item-Based Collaborative Filtering:

Calculate the similarity between items based on user ratings, using similar metrics.
    
3. Generating Recommendations
    
For User-Based Filtering:

Identify the most similar users to the target user.
Aggregate the ratings of these similar users for items the target user hasn’t rated yet.
Recommend the items with the highest aggregated ratings.
    
For Item-Based Filtering:
    
For each item that the user has rated, find similar items.
Aggregate the ratings for these similar items.
Recommend items that have the highest aggregated scores based on the user’s ratings.